<a href="https://colab.research.google.com/github/mariamwael10/Data-Classification/blob/main/DataClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Imports***

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ***Dataset***

In [11]:
columns = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym',  'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'Class']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data', names = columns)

feature_names = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym',  'fM3Long', 'fM3Trans', 'fAlpha', 'fDist']
X = data[feature_names]
display(data)

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,Class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


# ***Data Balancing and Splitting***
- value_counts() :Return a Series containing counts of unique rows in the DataFrame.
- np.random.choice(class , size, replace or not(replace :Default is True, meaning that a value of a can be selected multiple times.))

In [18]:
g = data[data.Class == 'g']
h = data[data.Class == 'h']
g_size = len(g)
h_size = len(h) 
difference = abs(g_size - h_size)

if g_size > h_size:
  extra_readings = np.random.choice(g.index, difference , replace=False)
  #update g after removing extra readings
  g = g.drop(extra_readings)

if h_size > g_size:
  extra_readings = np.random.choice(h.index, difference , replace=False)
  #update h after removing extra readings
  h = h.drop(extra_readings)

balanced_data = g.append(h)
display(balanced_data)
print(balanced_data['Class'].value_counts())

x = balanced_data[feature_names]
y = balanced_data['Class']
x_train , x_test , y_train , y_test = train_test_split(x , y, test_size = 0.3)
display(x_train)
print(x_train.shape, x_test.shape)

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,Class
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
5,51.6240,21.1502,2.9085,0.2420,0.1340,50.8761,43.1887,9.8145,3.6130,238.0980,g
8,96.2327,46.5165,4.1540,0.0779,0.0390,110.3550,85.0486,43.1844,4.8540,248.2260,g
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


g    6688
h    6688
Name: Class, dtype: int64


,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
7996,32.2136,17.0121,2.8995,0.3440,0.1834,9.4786,14.4098,-11.7009,0.5960,206.2050
11224,25.6039,16.1015,2.6799,0.3678,0.1912,-4.3692,9.8804,9.9835,10.2680,135.0660
12302,41.6637,13.8484,2.5410,0.3942,0.2403,42.8100,30.9380,10.1984,1.1540,224.6130
12349,146.4157,42.4736,2.9893,0.2665,0.2067,-160.4157,-8.5240,-29.3919,7.2265,232.5051
15606,43.1582,11.7072,3.0029,0.2996,0.1660,62.1722,25.3654,-14.3256,63.2298,198.9022
...,...,...,...,...,...,...,...,...,...,...
13628,99.6430,60.7624,3.1314,0.2627,0.1533,-38.2399,-74.2057,70.4102,12.6359,341.2119
14234,68.1675,17.6491,2.6488,0.4198,0.2144,-13.0659,-73.8798,12.2847,27.8760,178.3910
13804,45.6988,11.5450,2.5459,0.3807,0.2012,19.3990,15.2565,-2.4373,40.9735,189.3365
5801,84.6457,18.6030,2.5315,0.4294,0.2221,-42.1730,-52.6508,15.6937,2.2590,318.7300


(9363, 10) (4013, 10)
